In [23]:
import pandas as pd
pd.set_option('max_colwidth', 1000)
df = pd.read_json("017_after_kaldi_anchor_separated.jsonl", lines = True)


In [24]:
df.columns

Index(['file', 'start', 'end', 'asr_transcription', 'webpage_transcript',
       'ratio', 'text', 'kaldi_transcript', 'kaldi_words', 'kaldi_word_starts',
       'kaldi_word_ends'],
      dtype='object')

In [25]:
df[df.kaldi_transcript == ""].shape[0] / df.shape[0]

0.14510373193007922

We have 15% of instances where Kaldi could not find transcriptions

In [26]:
c = df[df.kaldi_transcript != ""]
(c.end - c.start ).sum()/60**2


69.10775000000001

The remaining parts make up for 68 hours of data. Let us see how the transcriptions look like and if they are stable:

In [33]:
df[["file","asr_transcription", "kaldi_transcript"]].sample(2)


,file,asr_transcription,kaldi_transcript
12366,audio/umlLa_kFcjk.wav,pa ja sam imao prilike a razgovorim sa svima dakle sješit ću sa mojim partnerima sa ljudima koji me dugo godina podržavaju sa nekim manjim grupama građenima koji su oko nas sa nevladinim organizacijama dakle čuti ću mišljenja pa ću vidjeti šta ljudi mislile da je najbolje zabiro pirot je jedn od rjetkih opština u kojoj je srpska napredna stranka opozicija sli vi ako kažete da se rađujete sa njima i pak su oni u poziciji i na lokalnom nivou često kritikuju vlast kako se rađujete sa njima kako se snalazite u taj situaciji,kimππ ja sam imao prilike da razgovaram sa svima sešću sa svojim partnerima sa ljudima koji me dugo godina podržavaju sa nekim manjim grupama građana koje su oko nas sa nevladinim organizacijama čuću mišljenja pa ću videti šta ljudi misle da je najbolje za pirotpi pirot je jedna od retkih opština u kojoj je sns opozicija iako kažete da sarađujete sa njima ipak su oni u opoziciji i na lokalnom nivou često kritikuju vlast kako sarađujete sa njima i kako sarađujete u toj situacijiππ
5828,audio/akVJcU0gZ7A.wav,kupanju u bezeni imaju klorzaisadobar jagdenseficijent da je hiv izuzetno osjetljiv upodešnjoj sredini da ne može ubodom odnosno ujednom hemotofakog insekta znači to je nešto što je ostalo,kupanjem u bazenima jer je hlor zaista dobar i jak dezificijent hiv je izuzetno osetljiv u spoljašnjoj sredini da ne može ubodom odnosno ujedom insekta to je nešto što je ostalo


Let's check if any files are missing:

In [28]:
df["segment_file"] = df.apply(lambda row:
    row["file"].replace(".wav", "").replace("audio", "seg_audio") + f'__{row["start"]:.1f}__{row["end"]:.1f}__.wav',
    axis=1
)

for file in df.segment_file:
    from pathlib import Path
    p = Path(file)
    if not p.exists():
        print("Segment", p, "does not exist!")



Optional step:

delete rows where we have empty kaldi outputs and delete corresponding audio files:

In [29]:
filtered_df = df[df.kaldi_transcript != ""]

loc = Path("seg_audio")
for segmentfile in loc.glob("*.wav"):
    if str(segmentfile) not in filtered_df["segment_file"]:
        # segmentfile.unlink()
        continue

Let's transform the string representations into something more pythonic:

In [34]:
from ast import literal_eval

filtered_df.loc[:, "kaldi_words"] = filtered_df.kaldi_words.apply(literal_eval)
filtered_df.loc[:, "kaldi_word_ends"] = filtered_df.kaldi_word_ends.apply(literal_eval)
filtered_df.loc[:, "kaldi_word_starts"] = filtered_df.kaldi_word_starts.apply(literal_eval)

filtered_df.loc[:, "kaldi_transcript"] = filtered_df.kaldi_transcript.str.replace("ππ", r"<\anchor>").str.replace("π", r"<anchor>")


/home/rupnik/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [35]:
filtered_df.to_json("018_juzne_vesti_anchor_separated.jsonl", lines=True, orient="records")

In [36]:
filtered_df.columns

Index(['file', 'start', 'end', 'asr_transcription', 'webpage_transcript',
       'ratio', 'text', 'kaldi_transcript', 'kaldi_words', 'kaldi_word_starts',
       'kaldi_word_ends', 'segment_file'],
      dtype='object')

In [37]:
filtered_df[['file', 'asr_transcription', "webpage_transcript",'kaldi_transcript']].head(30)

,file,asr_transcription,webpage_transcript,kaldi_transcript
0,audio/58xZSVbpgkk.wav,nekoliko puta smo ponavljali da nišo od uvođenja više stranačja nije ni kad imao veći broj narodnih poslanika u skupštini srbiji razgovaramo sa slavoljebom vlajkovićem poslanikom socijalistički partije srbije gospodine vlajkoviću kakav je vaš utesak da li je niž iskoristio to što ili koristi to što ima 12 poslanika,"Nekoliko puta smo ponavljali da Niš od uvođenja višestranačja nijenikada imao veći broj poslanika u Skupštini Srbije. Razgovaramo sa Slavoljubom Vlajkovićem, narodnim poslanikom Socijalističke partije Srbije. Gospodine Vlajkoviću, kakav je vaš utisak, da li je Niš iskoristio to što ili koristi to što ima 12 poslanika -",nekoliko puta smo ponavljali da niš od uvođenja višestranačja nijenikada imao veći broj poslanika u skupštini srbije razgovaramo sa slavoljubom vlajkovićem narodnim poslanikom socijalističke partije srbije gospodine vlajkoviću kakav je vaš utisak da li je niš iskoristio to što ili koristi to što ima dvanaest poslanika
1,audio/58xZSVbpgkk.wav,nikad više pa niti iskoristio niti će izgledati iskoristi tu priliku što mislim da je to pa problem je u nama mislim da je problem u gradu a nije problem u narodnim a mi jeste prestavo u svojo stranke i mi nismo bjereni kao lica mi smo ispred svojih stranaka iz,"- nikad više?’ Niti je iskoristio, niti će izgleda iskoristi priliku. Mislim da je to.. Šta je problem? Pa problem je u nama! Mislim da je problem u gradu, nije problem u narodnim poslanicima! Mi, jeste, predstavljamo svoje stranke! Mi nismo birani kao lica, mi smo ispred svojih stranaka",nikad više<nchor> niti je iskoristio niti će izgleda iskoristi priliku mislim da je topi šta je problem<nchor> pa problem je u nama mislim da je problem u gradu nije problem u narodnim poslanicima mi jeste predstavljamo svoje stranke mi nismo birani kao lica mi smo ispred svojih stranaka
2,audio/58xZSVbpgkk.wav,dobrani i znate da e svaka stranka od polih predloži 200 kandidata slučaj tako te o da smo svi bili na tim visokim brojevima da smo ušli u parlament ali mislim da je problem narodnih poslavnika ovdje u gradu,"Vi znate da svaka stranka otprilike predloži 250 kandidata . Slučaj je tako hteo da smo svi bili na tim visokim brojevima, da smo ušli u parlament. Ali mislim da je problem narodnih poslanika ovde u gradu!",vi znate da svaka stranka otprilike predloži dvijesto pedeset kandidata slučaj je tako hteo da smo svi bili na tim visokim brojevima da smo ušli u parlament ali mislim da je problem narodnih poslanika ovde u gradu
3,audio/58xZSVbpgkk.wav,znači grad uopštenije organizvan i mi faktički u gradu nemamo i jednu jedinu zajedničku želju koju bi trebali da pokušamo bar da rješimo i da skupštini iskoristimo za bolji niš i za bolje sud ta je problem zbog čega vi nemožete se a jesam,"gradu! Znači, grad uopšte nije organizovan i mi faktički u gradu nemamo ni jednu jedinu zajedničku želju koju bi trebalo da pokušamo bar da rešimo i da Skupštinu iskoristimo za bolji Niš i za bolje sutra. Ali šta je problem? Zbog čega vi ne možete da se",znači grad uopšte nije organizovan i mi faktički u gradu nemamo ni jednu jedinu zajedničku želju koju bi trebalo da pokušamo bar da rešimo i da skupštinu iskoristimo za bolji niš i za bolje sutrapi ali šta je problem zbog čega vi ne možete da se
4,audio/58xZSVbpgkk.wav,pa iima ali ja kažem da bi trebao grada kažem is to ja mogu individualno tako e socijalistička partije srvi ima najveći broj poslonika tu ste i zrano radovanović ivana dinić pa što se tiče nas kao socijaliste mislim da to radimo perfektno,"ja jesam... Pa ima, ali ja kažem da bi trebalo grad da kaže, nisu to.. Ja mogu individualno, ali tako mogu u ime svoje partije... Ali vi imate, Socijalistička partija ima najveći broj poslanika: tu ste vi, Zoran Radovanović i Ivana Dinić. Tako je! Pa, što se tiče nas socijalista, mislim da to",pa ima ali ja kažem da bi trebalo grad da kaže nisu to ja mogu individualno ali tako mogu u ime svoje partijep

In [38]:
corpus = pd.read_json("020_JV_with_metadata.jsonl", orient="records", lines=True)
corpus = corpus["path guest_name guest_description host".split()]
corpus.columns, filtered_df.columns

(Index(['path', 'guest_name', 'guest_description', 'host'], dtype='object'),
 Index(['file', 'start', 'end', 'asr_transcription', 'webpage_transcript',
        'ratio', 'text', 'kaldi_transcript', 'kaldi_words', 'kaldi_word_starts',
        'kaldi_word_ends', 'segment_file'],
       dtype='object'))

In [39]:
merged_df = pd.merge(
    left=filtered_df,
    right = corpus,
    how = "left",
    left_on="file",
    right_on="path"
)

merged_df = merged_df["file start end asr_transcription kaldi_transcript guest_name guest_description host kaldi_words kaldi_word_starts kaldi_word_ends segment_file".split()]

In [27]:
merged_df.to_json("021_JV_segments_with_metadata.jsonl", orient="records", lines=True)